In [7]:
import sqlite3
from sqlite3 import Error

In [8]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
        
    except Error as e:
        print(e)
    
    #finally:
        #if conn:
            #conn.close()



db = create_connection(r"database/wallet_db.db")

In [9]:
def execute_w_res(conn, query):
    try:
        c = conn.cursor()
        result = c.execute(query)
        return result.fetchall()
    except Error as e:
        print(e)
        return None

def execute(conn, query):
    try:
        c = conn.cursor()
        c.execute(query)
        return True
    except Error as e:
        print(e)
        return None

In [42]:
def create_wallet_table(conn):
    query1 =  f"CREATE TABLE IF NOT EXISTS Wallets ( xprv text NOT NULL PRIMARY KEY,\n "
    query2 = f"name text UNIQUE, words text NOT NULL) WITHOUT ROWID;"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def create_address_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Addresses ( address text NOT NULL PRIMARY KEY,\nacc_index INT NOT NULL,"
    query2 = "\npath text NOT NULL,\nchange_addr INT NOT NULL,\ncreated INT NOT NULL,\nwallet text NOT NULL,\nFOREIGN KEY (wallet) "
    query3 = "\nREFERENCES Wallets(xprv) ) WITHOUT ROWID ;"
    query = query1 + query2 + query3
    print(query)
    return execute(conn, query)

def create_utxo_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Utxos ( address text NOT NULL,\namount INT NOT NULL,\ntx_id text NOT NULL,"
    query2 = "\nout_index INT NOT NULL,\ncreated INT NOT NULL,\nspent INT NOT NULL,\nconfirmed INT NOT NULL, "
    query3 = "\nPRIMARY KEY (tx_id, out_index)\nFOREIGN KEY (address)\nREFERENCES Addresses(address) );"
    query = query1 + query2 + query3
    print(query)
    return execute(conn, query)

def create_transaction_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Transactions ( tx_id text NOT NULL PRIMARY KEY,\nlock_time INT,\nversion INT,\n"
    query2 = "\nconfirmations INT NOT NULL,\ncreated INT NOT NULL)  WITHOUT ROWID ;"
    query = query1 + query2
    print(query)
    return execute(conn, query)

def create_tx_in_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Tx_Ins ( tx_id text NOT NULL, out_index INT NOT NULL,\nspent_by text NOT NULL,\n"
    query2 = "FOREIGN KEY (tx_id,out_index)\nREFERENCES Utxo(tx_id,out_index) \n "
    query3 = "FOREIGN KEY (spent_by)\nREFERENCES Transactions(tx_id) )  WITHOUT ROWID ;"
    query = query1 + query2 + query3
    print(query)
    return execute(conn, query)

def create_tx_out_table(conn):
    query1 =f"CREATE TABLE IF NOT EXISTS Tx_Outs ( out_index INT NOT NULL,\n amount INT NOT NULL,\ncreated_by text NOT NULL,\n"
    query2 = "script_pubkey text NOT NULL, \n PRIMARY KEY (created_by, out_index)\n"
    query3 = "FOREIGN KEY (created_by)\nREFERENCES Transactions(tx_id) )  WITHOUT ROWID ;"
    query = query1 + query2 + query3
    print(query)
    return execute(conn, query)

def create_tables(conn):
    create_wallet_table(conn)
    create_address_table(conn)
    create_utxo_table(conn)
    create_transaction_table(conn)
    create_tx_in_table(conn)
    create_tx_out_table(conn)
    return True


def new_wallet(conn, xprv, words, name = None ):
    query1 = "INSERT INTO Wallets (xprv, words, name)\n "
    query2 = f"VALUES('{xprv}', '{words}', '{name}') ;"
    query = query1+query2
    print(query)
    return execute(conn, query)

def new_address(conn, address, path, acc_index, change_addr, created, wallet):
    query1 = "INSERT INTO Addresses (address, path, acc_index, change_addr, created, wallet)\n "
    query2 = f"VALUES({address}, {path}, {acc_index}, {change_addr}, {created}, {wallet}) ;"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def new_utxo(conn, address, amount, tx_id, out_index, created, spent=False, confirmed=False):
    query1 = "INSERT INTO Utxos (address, amount, tx_id, out_index, created, spent, confirmed)\n "
    query2 = f"VALUES({address}, {amount}, {tx_id}, {out_index}, {created}, {spent}, {confirmed});"
    query = query1+query2
    #print(query)
    return execute(conn, query)

def new_tx(conn, tx_id, tx_ins, tx_outs, lock_time=None, version=1 ):
    """
    tx_id: String. transaction id.
    tx_ins: List of touples: [ (prev_tx_id, index), ... ]
    tx_outs: List of touples: [ (out_index, amount, script_pubkey), ... ]
    lock_time: Int: transaction locktime.
    version: Int: version.
    """
    query1 = "INSERT INTO Transactions ( tx_id, lock_time, version)\n "
    query2 = f"VALUES({tx_id}, {lock_time}, {version});"
    query = query1+query2
    execute(conn, query)
    
    for tx_in in tx_ins:
        query3 = "INSERT INTO Tx_Ins ( tx_id, out_index, spent_by)\n "
        query4 = f"VALUES( {tx_in[0]}, {tx_in[1]}, {tx_id});"
        query = query3+query4
        execute(conn, query)
        
    for tx_out in tx_outs:
        query5 = "INSERT INTO Tx_Outs ( out_index, amount, script_pubkey, created_by)\n "
        query6 = f"VALUES( {tx_out[0]}, {tx_out[1]}, {tx_out[2]}, {tx_id});"
        query = query5+query6
        execute(conn, query)
    
    return execute(conn, query)




def does_table_exist(conn, table):
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}';"
    return execute_w_res(conn, query)

#def create_address(conn, address,i,change_addr,wallet_xprv):
    

In [40]:
res = create_tables(db)
res

CREATE TABLE IF NOT EXISTS Addresses ( address text NOT NULL PRIMARY KEY,
acc_index INT NOT NULL,
path text NOT NULL,
change_addr INT NOT NULL,
created INT NOT NULL,
wallet text NOT NULL,
FOREIGN KEY (wallet) 
REFERENCES Wallets(xprv) ) WITHOUT ROWID ;
CREATE TABLE IF NOT EXISTS Utxos ( address text NOT NULL,
amount INT NOT NULL,
tx_id text NOT NULL,
out_index INT NOT NULL,
created INT NOT NULL,
spent INT NOT NULL,
confirmed INT NOT NULL, 
PRIMARY KEY (tx_id, out_index)
FOREIGN KEY (address)
REFERENCES Addresses(address) );
CREATE TABLE IF NOT EXISTS Transactions ( tx_id text NOT NULL PRIMARY KEY,
lock_time INT,
version INT,

confirmations INT NOT NULL,
created INT NOT NULL)  WITHOUT ROWID ;
CREATE TABLE IF NOT EXISTS Tx_Ins ( tx_id text NOT NULL, out_index INT NOT NULL,
spent_by text NOT NULL,
FOREIGN KEY (tx_id,out_index)
REFERENCES Utxo(tx_id,out_index) 
 FOREIGN KEY (spent_by)
REFERENCES Transactions(tx_id) )  WITHOUT ROWID ;
PRIMARY KEY missing on table Tx_Ins
CREATE TABLE IF NOT 

True

In [17]:
res = create_address_table(db)

CREATE TABLE IF NOT EXISTS Addresses ( address text PRIMARY KEY,
acc_index INT NOT NULL UNIQUE,
change_addr INT NOT NULL,
created INT NOT NULL,
wallet text NOT NULL,
FOREIGN KEY (wallet) 
REFERENCES Wallets(xprv) ) WITHOUT ROWID ;


In [18]:
res

True

In [29]:
res = does_table_exist(db,"Wallets")

In [30]:
res

[]

In [25]:
f = db.cursor().execute("DROP TABLE Addresses")

In [6]:
db.close()

In [43]:
xprv ="tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP"
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
res = new_wallet(db, xprv, words)

INSERT INTO Wallets (xprv, words, name)
 VALUES('tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP', 'engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning', 'None') ;


In [44]:
res

True